<a href="https://colab.research.google.com/github/maryamstraykid/bigdata_project/blob/main/projet_bigdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installation de la biblithéqe pyspark
!pip3 install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 21.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=c63e6f3acc63076d9af9c0c2a3512d4c404b9c80c132dcb6b82579860a1771a1
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
from pyspark.context import SparkContext
#creation of SparkContext
sc = SparkContext.getOrCreate()

In [ ]:
from pyspark.sql import SparkSession
#creation of SparkSession
spark = SparkSession.builder.appName("commentaire hespress").getOrCreate()

In [ ]:
spark

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#read data and create dataframe
commentDF = spark.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load("/content/gdrive/My Drive/données_colab/comments_sport.csv")

In [ ]:
#prints out the schema in the tree format.
commentDF.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- postId: string (nullable = true)
 |-- comment: string (nullable = true)
 |-- score: string (nullable = true)
 |-- topic: string (nullable = true)



In [ ]:
#set data types - cast the data in columns to match schema
commentDF = commentDF .withColumn("postId", commentDF["postId"].cast("string")) \
  .withColumn("comment", commentDF["comment"].cast("string")) \
  .withColumn("score", commentDF["score"].cast("int")) \
  .withColumn("topic", commentDF["topic"].cast("string"))


In [ ]:
#prints out the new schema
commentDF.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- postId: string (nullable = true)
 |-- comment: string (nullable = true)
 |-- score: integer (nullable = true)
 |-- topic: string (nullable = true)



In [ ]:
#showing top 5 rows
commentDF.show(5)

+--------------------+--------------------+--------------------+-----+-----+
|                 _c0|              postId|             comment|score|topic|
+--------------------+--------------------+--------------------+-----+-----+
|                   0|af1ce6ec055c11eb9...|نهار الاربعاء الم...|  -34|sport|
|                   1|af1ce6ec055c11eb9...|كثرة المنتخبات وك...| null| null|
|لدينا رجاء عالمي ...|                null|                null| null| null|
|لدينا وداد أمة لط...|                null|                null| null| null|
|لدينا غزالة سوس ح...|                null|                null| null| null|
+--------------------+--------------------+--------------------+-----+-----+
only showing top 5 rows



Data analysis :

In [ ]:
from pyspark.sql.functions import col

In [ ]:
#counting the number of each score given
commentDF.groupBy("score").count().orderBy(col("count").desc()).show()

+-----+-----+
|score|count|
+-----+-----+
| null| 6535|
|    1|  316|
|    2|  313|
|    0|  310|
|    3|  257|
|    4|  228|
|    5|  211|
|    6|  200|
|    7|  196|
|   -1|  161|
|    9|  152|
|   11|  142|
|    8|  139|
|   10|  134|
|   -2|  116|
|   13|  108|
|   12|   98|
|   16|   88|
|   -3|   88|
|   14|   85|
+-----+-----+
only showing top 20 rows



In [ ]:
commentDF.groupBy("comment").avg("score").orderBy(col("avg(score)")).show(10)

+--------------------+----------+
|             comment|avg(score)|
+--------------------+----------+
|لدي خطة لو تم تطب...|      null|
|مستوى هابط جدا جد...|      null|
|وا هانتوما ياعباد...|      null|
|بيع مدافع ماشي مش...|      null|
|فريال لها ماضي و ...|      null|
|آش طلعنا من هاذ ا...|      null|
|دبا هاد الكعبي وا...|      null|
|تيقول المتل المغر...|      null|
|اتعجب لبعض التعال...|      null|
|خرج على راسو. كان...|      null|
+--------------------+----------+
only showing top 10 rows



Data cleaning & preprocessing for Machine learning:

> Bloc en retrait



In [ ]:
#select the important columns to attend our goal
to_select = ['score', 'comment']
df = commentDF.select(to_select)
df.show(5)

+-----+--------------------+
|score|             comment|
+-----+--------------------+
|  -34|نهار الاربعاء الم...|
| null|كثرة المنتخبات وك...|
| null|                null|
| null|                null|
| null|                null|
+-----+--------------------+
only showing top 5 rows



In [ ]:
#Returns a new DataFrame omitting rows with null values.
df = df.dropna(how='any')

In [ ]:
df = df.filter(df['score'] != 0)

In [ ]:
#Computes basic statistics for our columns.
df.describe().show()


+-------+------------------+--------------------+
|summary|             score|             comment|
+-------+------------------+--------------------+
|  count|              5964|                5964|
|   mean|13.420020120724345|                null|
| stddev| 51.64508007959286|                null|
|    min|              -639| c’est mon dernie...|
|    max|              1301|‏‎ ‎صوفاك معاملات...|
+-------+------------------+--------------------+



In [ ]:
#data type
df.dtypes

[('score', 'int'), ('comment', 'string')]

In [ ]:
def convert(x):
    if x > 0:
        return 'Good'
    else:
        return 'Bad'

In [ ]:
from pyspark.sql.types import *


In [ ]:
from pyspark.sql.functions import pandas_udf, PandasUDFType


In [ ]:
from pyspark.sql.functions import udf
#defining fuction that needs to register in the database library and use it on SQL as regular functions.
udfConvert = udf(convert, StringType())

In [ ]:
#creating of a new column "Sentiment"
df2 = df.withColumn("Sentiment", udfConvert('score'))

In [ ]:
#prints the new schema
df2.printSchema()

root
 |-- score: integer (nullable = true)
 |-- comment: string (nullable = true)
 |-- Sentiment: string (nullable = true)



In [ ]:
#show the new dataframe
df2.show()

+-----+--------------------+---------+
|score|             comment|Sentiment|
+-----+--------------------+---------+
|  -34|نهار الاربعاء الم...|      Bad|
|   16|احيي الاخ طاطا عل...|     Good|
|   17|منتخبات بمدربين و...|     Good|
|   -1|و ماذا عن منير ال...|      Bad|
|    4|شكرا على هذه المب...|     Good|
|   -2|السلام عليكم ورحم...|      Bad|
|   66|حمل القميص الوطني...|     Good|
| -120|هناك اسباب لا تري...|      Bad|
|  122|منتخب برواتب خيال...|     Good|
|   -5|لماذا كل هدا الحق...|      Bad|
|  -25|اصلا هذاك اللاعب ...|      Bad|
|   16|لا خير يرجى في من...|     Good|
|   30|المهم هو التركيز ...|     Good|
|   28|الحمدالله في الفط...|     Good|
|   12|والله الا دارو شي...|     Good|
|  -23|حمد الله لاعب كبي...|      Bad|
|    5|Hamdallah est seu...|     Good|
|   -8|غريب أمر هذا المد...|      Bad|
|  -16|حمد الله لاعب كبي...|      Bad|
|   12|لا اظن ان اللاعب ...|     Good|
+-----+--------------------+---------+
only showing top 20 rows



In [ ]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
# tokenizer
tokenizer = RegexTokenizer(inputCol="comment", outputCol="words")
# stop words remover
swRemover = StopWordsRemover(inputCol="words", outputCol="filtered")
# bag of words count
countVectors = CountVectorizer(inputCol="filtered", outputCol="features",
                              vocabSize=10000, minDF=5)

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
pipeline = Pipeline(stages=[tokenizer, swRemover, countVectors])
# Fit the pipeline to data
pipelineFit = pipeline.fit(df2)
data = pipelineFit.transform(df2)
data.show(5)

+-----+--------------------+---------+--------------------+--------------------+--------------------+
|score|             comment|Sentiment|               words|            filtered|            features|
+-----+--------------------+---------+--------------------+--------------------+--------------------+
|  -34|نهار الاربعاء الم...|      Bad|[نهار, الاربعاء, ...|[نهار, الاربعاء, ...|(5113,[1,3,5,11,1...|
|   16|احيي الاخ طاطا عل...|     Good|[احيي, الاخ, طاطا...|[احيي, الاخ, طاطا...|(5113,[1,2,3,4,15...|
|   17|منتخبات بمدربين و...|     Good|[منتخبات, بمدربين...|[منتخبات, بمدربين...|(5113,[2,3,17,65,...|
|   -1|و ماذا عن منير ال...|      Bad|[و, ماذا, عن, مني...|[و, ماذا, عن, مني...|(5113,[2,3,4,8,11...|
|    4|شكرا على هذه المب...|     Good|[شكرا, على, هذه, ...|[شكرا, على, هذه, ...|(5113,[1,2,3,15,3...|
+-----+--------------------+---------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
#counting our data
data.count()

5964

In [ ]:
#counting the data with good sentiment
data.filter(data['Sentiment']=='Good').count()

4546

In [ ]:
4546/5964


0.7622401073105298

so the majority of the data are positive . About 76% of scores are good ones.



In [ ]:
data2 = data.select(['Sentiment', 'features'])
data2.show(5)

+---------+--------------------+
|Sentiment|            features|
+---------+--------------------+
|      Bad|(5113,[1,3,5,11,1...|
|     Good|(5113,[1,2,3,4,15...|
|     Good|(5113,[2,3,17,65,...|
|      Bad|(5113,[2,3,4,8,11...|
|     Good|(5113,[1,2,3,15,3...|
+---------+--------------------+
only showing top 5 rows



In [ ]:
def convertNum(x):
    if x == 'Good':
        return 0
    else:
        return 1

In [ ]:
udfConvertNum = udf(convertNum, IntegerType())
#add new column label generated by the function convertNum
data3 = data2.withColumn("label", udfConvertNum('Sentiment'))

In [ ]:
data3.count()


5964

In [ ]:
from pyspark.sql.functions import *


In [ ]:
#specify the number of train data
train = data3.limit(5000)


In [ ]:
#save the rest as test data
test = data3.subtract(train)


Machine Learning :
Logistic Regression


In [ ]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model, only use features as input
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
lrModel = lr.fit(train)

In [ ]:
# Make predictions on test data using the transform() method.
# LogisticRegression.transform() will only use the 'features' column.
predictions = lrModel.transform(test)

In [ ]:
selected = predictions.select("label", "prediction", "probability")
display(selected)

DataFrame[label: int, prediction: double, probability: vector]

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

0.5506788333887802

Diagnostics & Model improvement :
Logistic Regression Grid Search Cross Validation :



In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())

In [ ]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(train)

In [ ]:
# Use test set to measure the accuracy of our model on new data
predictions2 = cvModel.transform(test)

In [ ]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions2)

0.5720696101262619